# 01 Visualize Signals（やさしいチュートリアル版）

このノートブックは、**EMA（移動平均）** と **ATR（ボラティリティ）** を使って、
「今はどんな地合いか？」「クロスのシグナルが出たか？」を順番に理解するためのミニ教科書です。  

> ゴール：読みながら動かし、パラメータをいじって「なぜこの判定になるか」を自分で説明できるようになる。

## Glossary（用語集）

- **OHLC**: 1本のローソク足を表す4つの価格。
  - **Open**（始値）: その期間の最初の価格
  - **High**（高値）: その期間で最も高い価格
  - **Low**（安値）: その期間で最も低い価格
  - **Close**（終値）: その期間の最後の価格
- **EMA（指数移動平均）**: 直近の値により重みを置いた平均。新しい情報に反応しやすいです。
- **EMA crossover（EMAクロスオーバー）**: 短期EMAが長期EMAを下から上に抜ける（または上から下に抜ける）現象。
  - 上抜けは上昇シグナル候補、下抜けは下降シグナル候補として使われます。
- **ATR（Average True Range）**: 値動きの大きさ（ボラティリティ）を示す指標。方向（上か下か）は示しません。
- **Regime（レジーム）**: 相場の「状態」。このノートでは ATR がしきい値より高い日を **Active regime**（活発）として扱います。

## 0) Parameter knobs（最初に調整するつまみ）

最初にこのセルだけ編集すれば、銘柄・期間・各種パラメータを一括で変更できます。

In [ ]:
# --- Parameter knobs: まずはここをいじる ---
SYMBOL = "1306.T"          # 例: "7203.T", "AAPL"
PERIOD = "2y"              # 取得期間（例: "6mo", "1y", "2y", "5y"）
INTERVAL = "1d"            # 足種（このノートでは日足を想定）

EMA_FAST_SPAN = 12          # 短期EMA
EMA_SLOW_SPAN = 26          # 長期EMA
ATR_PERIOD = 14             # ATR期間
THRESHOLD_WINDOW = 60       # ATRしきい値に使う移動中央値の窓
LAST_N_ROWS = 12            # 最後に表示する行数（学習用テーブル）

## 1) Data loading（データ取得）

In [ ]:
from quantlab.data import fetch_ohlc
from quantlab.indicators import ema, atr
from quantlab.plot import plot_price_ema, plot_atr_regime, plot_cross_points
import pandas as pd

# データ取得
# fetch_ohlc は通常 OHLCV を含むDataFrameを返します。
df = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL).copy()

# 指標計算（列名は後続セルでも使うため先に作成）
df["EMA_FAST"] = ema(df["Close"], EMA_FAST_SPAN)
df["EMA_SLOW"] = ema(df["Close"], EMA_SLOW_SPAN)
df["ATR"] = atr(df, ATR_PERIOD)

# EMA差分（クロス判定の基本）
df["ema_diff"] = df["EMA_FAST"] - df["EMA_SLOW"]

df.tail(3)

## 2) Data sanity check（データ健全性チェック）

分析に入る前に、次を確認します。
- 冒頭/末尾の行（head/tail）
- カラム一覧
- 日付範囲
- NaN 件数（指標計算の最初にNaNが出るのは自然）

In [ ]:
# 先頭と末尾を確認
print("=== df.head() ===")
display(df.head())
print("=== df.tail() ===")
display(df.tail())

# カラム一覧
print("\n=== columns ===")
print(df.columns.tolist())

# 日付範囲（DatetimeIndex想定。違う場合にも対応）
if isinstance(df.index, pd.DatetimeIndex):
    start_date = df.index.min()
    end_date = df.index.max()
else:
    # indexが日時でない場合は、Date列などを探すためのフォールバック
    start_date = df.index.min()
    end_date = df.index.max()
print("\n=== date range ===")
print(f"start: {start_date}")
print(f"end  : {end_date}")

# NaN 件数
print("\n=== NaN counts ===")
print(df.isna().sum())

## 3) Explain EMA and ATR with numbers（数値で理解する）

最新日と前日を並べて、EMAとATRの変化を具体的に見ます。

In [ ]:
# 最新日・前日を取り出して、学習しやすい形で表示
latest = df.iloc[-1]
prev = df.iloc[-2]

print("=== latest / previous values ===")
print(f"date(latest): {df.index[-1]}")
print(f"date(prev)  : {df.index[-2]}")
print(f"EMA{EMA_FAST_SPAN} latest: {latest['EMA_FAST']:.4f}")
print(f"EMA{EMA_FAST_SPAN} prev  : {prev['EMA_FAST']:.4f}")
print(f"EMA{EMA_SLOW_SPAN} latest: {latest['EMA_SLOW']:.4f}")
print(f"EMA{EMA_SLOW_SPAN} prev  : {prev['EMA_SLOW']:.4f}")
print(f"ATR{ATR_PERIOD} latest  : {latest['ATR']:.4f}")
print(f"ATR{ATR_PERIOD} prev    : {prev['ATR']:.4f}")

ema_diff_today = latest["ema_diff"]
ema_diff_prev = prev["ema_diff"]
print("\n=== ema_diff ===")
print(f"ema_diff_today: {ema_diff_today:.4f}")
print(f"ema_diff_prev : {ema_diff_prev:.4f}")

## 4) Explain Active regime（アクティブ判定を理解する）

ここでは、
- `ATR` の **THRESHOLD_WINDOW日移動中央値** をしきい値にする
- `ATR > threshold` の日を `active=True` とする

というルールを使います。

In [ ]:
# ATRのしきい値（移動中央値）
df["atr_threshold"] = df["ATR"].rolling(THRESHOLD_WINDOW).median()

# Active regime の真偽
df["active"] = df["ATR"] > df["atr_threshold"]

latest_atr = df["ATR"].iloc[-1]
latest_threshold = df["atr_threshold"].iloc[-1]
latest_active = bool(df["active"].iloc[-1])

print("=== Active regime (latest) ===")
print(f"ATR{ATR_PERIOD} latest : {latest_atr:.4f}")
print(f"median{THRESHOLD_WINDOW} threshold: {latest_threshold:.4f}")
print(f"active: {latest_active}")

active_ratio = df["active"].mean() * 100
print(f"\n% of days active in dataset: {active_ratio:.2f}%")

## 5) Explain crossover signals（クロスシグナルを理解する）

クロスの基本ロジック：
- `crossed_up`: 前日まで `ema_diff <= 0` で当日 `ema_diff > 0`
- `crossed_down`: 前日まで `ema_diff >= 0` で当日 `ema_diff < 0`

このノートでは、`active=True` のときだけ最終シグナルを出します。

In [ ]:
# Boolean logic を明示
crossed_up = (df["ema_diff"].shift(1) <= 0) & (df["ema_diff"] > 0)
crossed_down = (df["ema_diff"].shift(1) >= 0) & (df["ema_diff"] < 0)

# active フィルタ込みのシグナル
#   +1: BUY候補（上抜け）
#   -1: SELL候補（下抜け）
#    0: シグナルなし
df["signal"] = 0
df.loc[crossed_up & df["active"], "signal"] = 1
df.loc[crossed_down & df["active"], "signal"] = -1

print("=== logic preview (last 5) ===")
display(
    pd.DataFrame({
        "ema_diff_prev<=0": (df["ema_diff"].shift(1) <= 0),
        "ema_diff_today>0": (df["ema_diff"] > 0),
        "crossed_up": crossed_up,
        "ema_diff_prev>=0": (df["ema_diff"].shift(1) >= 0),
        "ema_diff_today<0": (df["ema_diff"] < 0),
        "crossed_down": crossed_down,
    }).tail(5)
)

# 学習用の最終テーブル
cols = ["Close", "EMA_FAST", "EMA_SLOW", "ema_diff", "ATR", "active", "signal"]
print(f"\n=== last {LAST_N_ROWS} rows ===")
display(df[cols].tail(LAST_N_ROWS))

## 6) Plot 1: Price + EMA

価格（Close）と短期/長期EMAを重ねると、
- トレンド方向
- EMA同士の距離（勢い）
を視覚的に確認できます。

In [ ]:
# 既存プロット関数をそのまま利用
plot_price_ema(df)

## 7) Plot 2: ATR regime

ATRとしきい値の関係から、
「今は値動きが活発かどうか（active）」を把握します。

In [ ]:
plot_atr_regime(df)

## 8) Plot 3: Cross points

クロス発生点（上抜け/下抜け）を可視化します。  
`active` フィルタと合わせて見ると、シグナルの選別理由が分かりやすくなります。

In [ ]:
plot_cross_points(df)